# Imports

In [53]:
import random
import copy
import math
import matplotlib.pyplot as plt


# Graph dicts

In [54]:
# A dictionary mapping graph filenames to their known chromatic numbers
GRAPH_DICT = {
    #'myciel3.col': 4,
    'queen8_8.col': 9,
    #'le450_15b.col': 15
}
#PRINT_EACH = 50  # Number of generations between print statements
CHROMATIC_MARGIN = 5 # Margin above known chromatic number to start with

# Declaring Functions

## Parse graph from .col file (Dataset)

In [55]:
def parse_col_file(filepath):
    adj = {}
    num_vertices = 0

    with open(filepath, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('p'):
                _, _, n, _ = line.split()
                num_vertices = int(n)
                adj = {i: set() for i in range(1, num_vertices + 1)}

            elif line.startswith('e'):
                _, u, v = line.split()
                u, v = int(u), int(v)
                adj[u].add(v)
                adj[v].add(u)
    return adj, num_vertices


## Fitness and Conflict Evaluation

In [56]:
def count_conflicts(adj, coloring):
    conflicts = 0
    for u in adj:
        for v in adj[u]:
            if u < v and coloring[u] == coloring[v]:
                conflicts += 1
    return conflicts

In [57]:
def num_colors_used(coloring):
    return len(set(coloring.values()))

In [58]:
def fitness(adj, coloring, alpha=1, beta=1):
    return alpha * count_conflicts(adj, coloring) + beta * num_colors_used(coloring)

## Population Init

In [59]:
def random_coloring(num_vertices, max_colors):
    return {i: random.randint(1, max_colors) for i in range(1, num_vertices + 1)}

In [60]:
def initialize_population(pop_size, num_vertices, max_colors):
    return [random_coloring(num_vertices, max_colors) for _ in range(pop_size)]

## Tournament Selection

In [61]:
def tournament_selection(population, adj, k=3):
    candidates = random.sample(population, k)
    return min(candidates, key=lambda c: fitness(adj, c))

## Crossover (Uniform)

In [62]:
def uniform_crossover(parent1, parent2):
    child = {}
    for v in parent1:
        child[v] = parent1[v] if random.random() < 0.5 else parent2[v]
    return child


## Mutation and conflict fix

In [63]:
def mutate(coloring, adj, max_colors, mutation_rate=0.02):
    for v in coloring:
        if random.random() < mutation_rate:
            coloring[v] = random.randint(1, max_colors)

    # Extra: fix conflicts
    for u in adj:
        for v in adj[u]:
            if coloring[u] == coloring[v]:
                coloring[v] = random.randint(1, max_colors)

# Genetic Algorithm

## Definition of the Genetic Function

In [64]:
def genetic_algorithm(
    adj,
    num_vertices,
    max_colors,
    pop_size=100,
    generations=1000,
    elitism_rate=0.05,
    mutation_rate=0.02
):
    chromatic_number = max_colors - CHROMATIC_MARGIN   # Chromatic number 
    if chromatic_number < 5:
        print_each = 1
    if chromatic_number >= 5 and chromatic_number < 10:
        print_each = 10
    if chromatic_number >= 10 and chromatic_number < 20:
        print_each = 50
    population = initialize_population(pop_size, num_vertices, max_colors) # Initial population
    elite_size = int(pop_size * elitism_rate) # Number of elites
    # initialize best solution tracking
    best_solution = None
    best_fitness = float('inf')
    found_feasible = False
    conflicts = []
    
    # main GA loop
    for gen in range(generations):
        population.sort(key=lambda c: fitness(adj, c))  # Sort by fitness
        elites = population[:elite_size]               # Select elites

        current_best = population[0]
        current_fitness = fitness(adj, current_best)
        current_conflicts = count_conflicts(adj, current_best)
        current_colors = num_colors_used(current_best)

        # Update global best
        if current_fitness < best_fitness:                  # New best found
            best_solution = copy.deepcopy(current_best)     # Update best solution
            best_fitness = current_fitness                  # Update best fitness

        if gen % print_each == 0:   # Print every set number of generations
            conflicts.append(current_conflicts)
            print(
                f"Gen {gen} | Best fitness: {best_fitness} | "
                f"Conflicts: {count_conflicts(adj, best_solution)} | "
                f"Colors: {num_colors_used(best_solution)}"
            )

        # First feasible solution (any number of colors)
        if current_conflicts == 0 and not found_feasible:
            print(f"Feasible solution found at generation {gen}")
            found_feasible = True

        # Early stopping condition
        if current_conflicts == 0 and current_colors <= chromatic_number:   # Optimal solution found, colors equal chromatic number and no conflicts
            print(
                f"Optimal solution found at generation {gen} | "
                f"Colors: {current_colors}"
            )
            return current_best, conflicts

        new_population = elites.copy()  # Start new population with elites (unchanged)

        while len(new_population) < pop_size:   # Fill the rest of the population using tournament selection, crossover, and mutation
            p1 = tournament_selection(population, adj)
            p2 = tournament_selection(population, adj)

            child = uniform_crossover(p1, p2)
            mutate(child, adj, max_colors, mutation_rate)

            new_population.append(child)

        population = new_population

    return best_solution, conflicts


## Usage of the Genetic Algorithm

In [65]:
if __name__ == "__main__":
    for filename, chromatic_number in GRAPH_DICT.items():
        adj, n = parse_col_file(filename)

        max_colors = chromatic_number + CHROMATIC_MARGIN

        print("=" * 55)
        print(f"{filename}: n={n}, chromatic_number={chromatic_number} max_colors={max_colors}")
        print("=" * 55)

        solution, conflicts = genetic_algorithm(
            adj=adj,
            num_vertices=n,
            max_colors=max_colors,
            pop_size=200,
        )

        print("=" * 50)
        print("Final solution:")
        print("Conflicts:", count_conflicts(adj, solution))
        print("Colors used:", num_colors_used(solution))    
        print("=" * 50)



queen8_8.col: n=64, chromatic_number=9 max_colors=14
Gen 0 | Best fitness: 49 | Conflicts: 35 | Colors: 14
Gen 10 | Best fitness: 21 | Conflicts: 7 | Colors: 14
Gen 20 | Best fitness: 21 | Conflicts: 7 | Colors: 14
Gen 30 | Best fitness: 20 | Conflicts: 6 | Colors: 14
Gen 40 | Best fitness: 18 | Conflicts: 4 | Colors: 14
Gen 50 | Best fitness: 18 | Conflicts: 4 | Colors: 14
Gen 60 | Best fitness: 18 | Conflicts: 4 | Colors: 14
Gen 70 | Best fitness: 18 | Conflicts: 4 | Colors: 14
Gen 80 | Best fitness: 18 | Conflicts: 4 | Colors: 14
Gen 90 | Best fitness: 16 | Conflicts: 2 | Colors: 14
Gen 100 | Best fitness: 16 | Conflicts: 2 | Colors: 14
Gen 110 | Best fitness: 16 | Conflicts: 2 | Colors: 14
Gen 120 | Best fitness: 16 | Conflicts: 2 | Colors: 14
Gen 130 | Best fitness: 16 | Conflicts: 2 | Colors: 14
Gen 140 | Best fitness: 15 | Conflicts: 1 | Colors: 14
Gen 150 | Best fitness: 15 | Conflicts: 1 | Colors: 14
Gen 160 | Best fitness: 15 | Conflicts: 1 | Colors: 14
Gen 170 | Best fitness

# Tabu Search

## Implementation of an optimized conflict storage
Using a data structure reduces the amount of computations since the tabu search requires not only the amount of conflicts but where have they appeared in order to generate meaningful neighborhood moves, because only recoloring conflicted vertices can reduce conflicts.
The total conflict count alone provides no structural information about where the problem is, making targeted local search impossible.

In [66]:
def conflicting_vertices(adj, coloring):
    conflicts = set()
    for u in adj:
        for v in adj[u]:
            if u < v and coloring[u] == coloring[v]:
                conflicts.add(u)
                conflicts.add(v)
    return list(conflicts)


## Implementation of the algorithm

In [67]:
def tabu_search(
    adj,
    num_vertices,
    max_colors,
    max_iterations=20000,
    tabu_tenure=7,
    aspiration=True,
):
    chromatic_number = max_colors - CHROMATIC_MARGIN   # Chromatic number
    if chromatic_number < 5:
        print_each = 1
    if chromatic_number >= 5 and chromatic_number < 10:
        print_each = 10
    if chromatic_number >= 10 and chromatic_number < 20:
        print_each = 50
    current = random_coloring(num_vertices, max_colors)
    current_conflicts = count_conflicts(adj, current)
    current_colors = num_colors_used(current)

    best = copy.deepcopy(current)
    best_conflicts = current_conflicts
    best_colors = current_colors
    found_optimal = False
    tabu_list = {}
    iteration = 0

    while iteration < max_iterations and not found_optimal:

        conflicted = conflicting_vertices(adj, current)
        best_move = None
        best_score = (float("inf"), float("inf"))

        for v in conflicted:
            original_color = current[v]

            for c in range(1, max_colors + 1):
                if c != original_color:

                    current[v] = c
                    new_conflicts = count_conflicts(adj, current)
                    new_colors = num_colors_used(current)

                    score = (new_conflicts, new_colors)
                    delta_better = score < best_score

                    move = (v, c)
                    tabu = move in tabu_list and tabu_list[move] > iteration

                    if tabu and aspiration:
                        if (new_conflicts < best_conflicts or
                           (new_conflicts == best_conflicts and new_colors < best_colors)):
                            tabu = False

                    if not tabu and delta_better:
                        best_score = score
                        best_move = (v, c, new_conflicts, new_colors)

                    current[v] = original_color

        if best_move is not None:
            v, c, new_conflicts, new_colors = best_move
            old_color = current[v]

            current[v] = c
            current_conflicts = new_conflicts
            current_colors = new_colors

            tabu_list[(v, old_color)] = iteration + tabu_tenure

            # Update global best using lexicographic objective
            if (current_conflicts < best_conflicts or
               (current_conflicts == best_conflicts and current_colors < best_colors)):
                best = copy.deepcopy(current)
                best_conflicts = current_conflicts
                best_colors = current_colors

        if iteration % print_each == 0:
            print(
                f"Iter {iteration} | "
                f"Conflicts: {current_conflicts} | "
                f"Colors: {current_colors} | "
                f"Best colors: {best_colors}"
            )
        # Early stopping condition
        if best_conflicts == 0 and best_colors <= chromatic_number:
            print(
                f"Optimal solution found at iteration {iteration} | "
                f"Colors: {best_colors}"
            )
            found_optimal = True

        iteration += 1

    return best


## Usage of Tabu Search

In [68]:
if __name__ == "__main__":
    for filename, chromatic_number in GRAPH_DICT.items():
        adj, n = parse_col_file(filename)

        max_colors = chromatic_number + CHROMATIC_MARGIN

        print("=" * 50)
        print(f"{filename}: n={n}, chromatic_number={chromatic_number} max_colors={max_colors}")
        print("=" * 50)

        solution = tabu_search(
            adj=adj,
            num_vertices=n,
            max_colors=max_colors,
            max_iterations=2000,
            tabu_tenure=7,
        )
        print("=" * 50)
        print("Final solution:")
        print("Conflicts:", count_conflicts(adj, solution))
        print("Colors used:", num_colors_used(solution))    
        print("=" * 50)


queen8_8.col: n=64, chromatic_number=9 max_colors=14
Iter 0 | Conflicts: 58 | Colors: 14 | Best colors: 14
Iter 10 | Conflicts: 23 | Colors: 14 | Best colors: 14
Iter 20 | Conflicts: 5 | Colors: 14 | Best colors: 14
Iter 30 | Conflicts: 0 | Colors: 14 | Best colors: 14
Iter 40 | Conflicts: 0 | Colors: 14 | Best colors: 14
Iter 50 | Conflicts: 0 | Colors: 14 | Best colors: 14
Iter 60 | Conflicts: 0 | Colors: 14 | Best colors: 14
Iter 70 | Conflicts: 0 | Colors: 14 | Best colors: 14
Iter 80 | Conflicts: 0 | Colors: 14 | Best colors: 14
Iter 90 | Conflicts: 0 | Colors: 14 | Best colors: 14
Iter 100 | Conflicts: 0 | Colors: 14 | Best colors: 14
Iter 110 | Conflicts: 0 | Colors: 14 | Best colors: 14
Iter 120 | Conflicts: 0 | Colors: 14 | Best colors: 14
Iter 130 | Conflicts: 0 | Colors: 14 | Best colors: 14
Iter 140 | Conflicts: 0 | Colors: 14 | Best colors: 14
Iter 150 | Conflicts: 0 | Colors: 14 | Best colors: 14
Iter 160 | Conflicts: 0 | Colors: 14 | Best colors: 14
Iter 170 | Conflicts:

# Simmulated Annealing

## Algorithm implementation

In [69]:
def simulated_annealing(
    adj,
    num_vertices,
    max_colors,
    vert_list,
    max_iterations=50000,
    initial_temp=10.0,
    cooling_rate=0.9995,
    conflict_weight=1000,
):
    chromatic_number = max_colors - CHROMATIC_MARGIN   # Chromatic number
    if chromatic_number < 5:
        print_each = 100
    if chromatic_number >= 5 and chromatic_number < 10:
        print_each = 500
    if chromatic_number >= 10 and chromatic_number < 20:
        print_each = 1000
    current = random_coloring(num_vertices, max_colors)
    current_conflicts = count_conflicts(adj, current)
    current_colors = num_colors_used(current)
    found_optimal = False
    best = copy.deepcopy(current)
    best_conflicts = current_conflicts
    best_colors = current_colors

    temperature = initial_temp
    iteration = 0

    while (iteration < max_iterations and temperature > 1e-6) and not found_optimal:

        # Pick a vertex biased toward conflicts
        conflicted = conflicting_vertices(adj, current)
        if len(conflicted) > 0:
            v = random.choice(conflicted)
        else:
            v = random.choice(vert_list)

        original_color = current[v]
        new_color = random.randint(1, max_colors)

        if new_color != original_color:

            current[v] = new_color
            new_conflicts = count_conflicts(adj, current)
            new_colors = num_colors_used(current)

            delta = (
                (new_conflicts - current_conflicts) * conflict_weight
                + (new_colors - current_colors)
            )

            accept = False
            if delta <= 0:
                accept = True
            else:
                prob = math.exp(-delta / temperature)
                accept = random.random() < prob

            if accept:
                current_conflicts = new_conflicts
                current_colors = new_colors

                if (
                    current_conflicts < best_conflicts or
                    (current_conflicts == best_conflicts and current_colors < best_colors)
                ):
                    best = copy.deepcopy(current)
                    best_conflicts = current_conflicts
                    best_colors = current_colors
            else:
                current[v] = original_color

        temperature *= cooling_rate
        iteration += 1

        if iteration % print_each == 0:
            print(
                f"Iter {iteration} | "
                f"T={temperature:.4f} | "
                f"Conflicts: {current_conflicts} | "
                f"Colors: {current_colors} | "
                f"Best colors: {best_colors}"
            )
        # Early stopping condition
        if best_conflicts == 0 and best_colors <= chromatic_number:
            print(
                f"Optimal solution found at iteration {iteration} | "
                f"Colors: {best_colors}"
            )
            found_optimal = True

    return best


## Usage of Simulated Annealing

In [70]:
if __name__ == "__main__":
    for filename, chromatic_number in GRAPH_DICT.items():
        adj, n = parse_col_file(filename)
        vert_list = list(adj.keys())

        max_colors = chromatic_number + CHROMATIC_MARGIN

        print("=" * 50)
        print(f"{filename}: n={n}, chromatic_number={chromatic_number} max_colors={max_colors}")
        print("=" * 50)

        solution = simulated_annealing(
            adj=adj,
            num_vertices=n,
            max_colors=max_colors,
            vert_list=vert_list
        )

        print("=" * 50)
        print("Final solution:")
        print("Conflicts:", count_conflicts(adj, solution))
        print("Colors used:", num_colors_used(solution))    
        print("=" * 50)   


queen8_8.col: n=64, chromatic_number=9 max_colors=14
Iter 500 | T=7.7875 | Conflicts: 0 | Colors: 14 | Best colors: 14
Iter 1000 | T=6.0645 | Conflicts: 0 | Colors: 14 | Best colors: 14
Iter 1500 | T=4.7228 | Conflicts: 0 | Colors: 14 | Best colors: 14
Iter 2000 | T=3.6779 | Conflicts: 0 | Colors: 14 | Best colors: 14
Iter 2500 | T=2.8642 | Conflicts: 0 | Colors: 14 | Best colors: 14
Iter 3000 | T=2.2305 | Conflicts: 0 | Colors: 14 | Best colors: 14
Iter 3500 | T=1.7370 | Conflicts: 0 | Colors: 14 | Best colors: 14
Iter 4000 | T=1.3527 | Conflicts: 0 | Colors: 14 | Best colors: 14
Iter 4500 | T=1.0534 | Conflicts: 0 | Colors: 14 | Best colors: 14
Iter 5000 | T=0.8203 | Conflicts: 0 | Colors: 14 | Best colors: 14
Iter 5500 | T=0.6388 | Conflicts: 0 | Colors: 14 | Best colors: 14
Iter 6000 | T=0.4975 | Conflicts: 0 | Colors: 14 | Best colors: 14
Iter 6500 | T=0.3874 | Conflicts: 0 | Colors: 14 | Best colors: 14
Iter 7000 | T=0.3017 | Conflicts: 0 | Colors: 14 | Best colors: 14
Iter 7500 